<a href="https://colab.research.google.com/github/Imvicoder/VoiceBotDemo/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q openai-whisper
# !pip install -q gradio
# !apt install ffmpeg
# !pip install TTS
# !pip install -q -U google-generativeai
!!pip install ffmpeg-python
!pip install langchain-google-genai # Installs the necessary package for using Google Gemini with LangChain.
!pip install gtts
!pip install pydub

In [2]:
import numpy as np
import whisper
from IPython.display import display, HTML
from google.colab.output import eval_js
from base64 import b64decode
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
from scipy.io.wavfile import write

from gtts import gTTS
from IPython.display import Audio
from pydub import AudioSegment
from pydub.playback import play
from threading import Thread
from time import sleep

In [3]:
import torch
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU")


# Load model and move to GPU if available
model = whisper.load_model("medium").to(device)

GPU is not available, using CPU


100%|█████████████████████████████████████| 1.42G/1.42G [00:28<00:00, 54.3MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [19]:
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    //var preview = document.createElement('audio');  # this is used to play the recorded audio
    //preview.controls = true;
    //preview.src = url;
    //document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = ""
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""


In [20]:
def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [5]:
# # Function to save numpy array as WAV file
# def save_audio_as_wav(audio_np, sample_rate, filename="recorded_audio.wav"):
#     write(filename, sample_rate, audio_np)

In [6]:
# Function to transcribe audio using Whisper
def transcribe_audio():
    audio_np, sample_rate = get_audio()
    filename = "recorded_audio.wav"
    write(filename, sample_rate, audio_np)
    # save_audio_as_wav(audio, sr)
    result = model.transcribe(filename)
    return result['text']

In [21]:
class InvisibleAudio(Audio):

    def _repr_html_(self):
        audio = super()._repr_html_()
        audio = audio.replace(
            '<audio',
            (
                '<audio onended="'
                + hide_closest(css_class=OUTPUT_AREA_CLASS)
                + '" onloadstart="' + hide_closest(css_class=OUTPUT_AREA_CLASS) + '"'
            )
        )
        return f'<div style="display:none">{audio}</div>'

OUTPUT_AREA_CLASS = 'jp-OutputArea-child'
OUT_CLASS = 'jp-transient-html'

In [22]:
def try_to_hide_parent(css_class, level):
    """deprecated"""
    return (
        f'var parent = this{".parentNode" * level};'
        f'if(parent.className == \'{css_class}\') parent.style.display = \'none\';'
    )


In [23]:

def hide_closest(css_class):
    return (
        f"var widget_our = this.closest('.{OUT_CLASS}');"
        f"var that = widget_our ? widget_our : this;"
        f"that.closest('.{css_class}').style.display = 'none';"
    )


In [24]:
def delete_closest(css_class):
    return (
        f"var widget_our = this.closest('.{OUT_CLASS}');"
        f"var that = widget_our ? widget_our : this;"
        f"var el = that.closest('.{css_class}'); el.parentNode.removeChild(el);"
    )


In [25]:
def delete_element(levels_up=1):
    return (
        'this'
        + '.parentNode' * levels_up
        + '.removeChild('
          + 'this'
          + '.parentNode' * (levels_up - 1)
        + ')'
    )

In [26]:
class InvisibleAudio(Audio):

    def _repr_html_(self):
        audio = super()._repr_html_()
        audio = audio.replace(
            '<audio',
            (
                '<audio onended="'
                + hide_closest(css_class=OUTPUT_AREA_CLASS)
                + '" onloadstart="' + hide_closest(css_class=OUTPUT_AREA_CLASS) + '"'
            )
        )
        return f'<div style="display:none">{audio}</div>'

In [27]:
def javascript_in_jupyter(code):
    return f'<img onerror="{code}" src="x" style="display:none"></div>'


def delayed_close(out, element, delay):
    sleep(delay)
    element.close()
    out.clear_output()
    out.close()


In [28]:
def transient_html(code: str, lifetime: float = 3):
    """Attempts to display and then hide given HTML code.

    If the hiding is not possible (no ipywidgets) fall-backs to persistent display.

    Arguments:
        code: HTML code as a string
        lifetime: time
    """
    try:
        from ipywidgets import HTML
        from ipywidgets.widgets import Output
        element = HTML(code)
        out = Output()
        out.add_class(OUT_CLASS)

        thread = Thread(target=delayed_close, args=(out, element, lifetime))

        with out:
            display(element)

        display(out)

        thread.start()

    except ImportError:
        from IPython.display import HTML
        element = HTML(code)
        display(element)

In [29]:
def scroll_to_current_cell(
    scroll_behaviour='smooth',
    preserve=False,
    smart=True,
    scroll_to='center',
    cell_class='jp-Cell'
):
    behaviour = f"behavior: '{scroll_behaviour}', block: '{scroll_to}'"
    code = (
        # scroll to the output
        '\n'.join([
            f"let cell = this.closest('.{cell_class}');",
            'let rect = cell.getBoundingClientRect();',
            (
                (
                    'let is_visible = rect.top >= 0 &&'
                    ' rect.bottom <= (window.innerHeight || document.documentElement.clientHeight);'
                )
                if smart else
                'let is_visible = false;'
            ),
            'if (!is_visible) {',
            '    cell.scrollIntoView({' + behaviour + '});',
            '}'
        ])
        # hide the output generated as an anchor
        + hide_closest(css_class=OUTPUT_AREA_CLASS)
    )
    code = javascript_in_jupyter(code)

    if not preserve:
        transient_html(code)

    if preserve:
        from IPython.display import HTML
        element = HTML(code)
        display(element)


In [10]:
def play_sound(filename):
    audio = InvisibleAudio(filename=filename, autoplay=True)
    transient_html(audio._repr_html_())

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

from langchain_core.prompts import ChatPromptTemplate

In [12]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [13]:
transcription = transcribe_audio()

print("Transcription:")
print(transcription)

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription:
 My name is Vikas, I'm calling from UnitedHealthcare. Can we connect?


In [14]:
def text_to_speech(text):
    # Convert text to speech
    tts = gTTS(text=text, lang='en', tld='com', slow=False)
    tts.save("output.wav")
    play_sound("output.wav")

# Example usage
text_to_speech("sure, what is this regarding for ?")

Output(_dom_classes=('jp-transient-html',))

In [15]:
def get_llm_output(prompt, text):
    parser = StrOutputParser()
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0)
    llm_chain = prompt | llm | parser
    out = llm_chain.invoke({"input": text})
    return out

In [16]:
extract_name_prompt_msg = f''' As an AI assitant, you are given a sentence, extract the name of the speaker. Strictly return me the name of speaker.
                        '''
extract_name_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", extract_name_prompt_msg),
        ("human", "{input}"),
    ]
)
extract_dob_msg = f''' As an AI assitant, you are given a sentence, extract the date of birth of the speaker. Strictly return me the date of birth of speaker in mm/dd/yyyy format.
                        '''
extract_dob_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", extract_dob_msg),
        ("human", "{input}"),
    ]
)
extract_id_msg = f''' As an AI assitant, you are given a sentence, extract the ID of the speaker. Strictly return me the ID of speaker.
                        '''
extract_id_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", extract_id_msg),
        ("human", "{input}"),
    ]
)

intent_classifier_msg = f''' You are given a text from a user. As an AI assistant, Help me in identifying the intent of the text. Strictly classify the
                            text into 4 categories. 1) reschedule 2) schedule 3) cancellation 4) other . Strictly classify the text frobased on these 4 categories and strictly return me 1 of these categories based on the text.
                        '''

intent_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", intent_classifier_msg),
        ("human", "{input}"),
    ]
)

member_availability_msg = f''' You are given a text from a user. As an AI assistant, Help me in identifying whether member is available on
                        '''

intent_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", intent_classifier_msg),
        ("human", "{input}"),
    ]
)
def get_intent(text):
    intent = get_llm_output(intent_prompt, text)
    print(f"Extracted intent: {intent}")
    return intent

def get_name(text):
    name = get_llm_output(extract_name_prompt, text)
    # print(f"Extracted name: {name}")
    return name

def get_dob(text):
    dob = get_llm_output(extract_dob_prompt, text)
    # print(f"Extracted DOB: {dob}")
    return dob

def get_id(text):
    member_id = get_llm_output(extract_id_prompt, text)
    # print(f"Extracted ID: {member_id}")
    return member_id

In [17]:
get_name('my name is vikas dixit')

'vikas dixit'

In [18]:
patient_details_dict = {
                        '1hb1':
                            {  'name': 'vikas',
                                'dob': '08/02/1995',
                            },
                        '1hb2':
                            {   'name': 'sachin',
                                'dob': '03/01/1995',
                            },
                        '1hb3':
                            {   'name': 'john',
                                'dob': '05/08/1990',
                            }
                        }

available_dates = {
                    '1hb1': ['Jan 10, 2025', 'Jan 17, 2025'],
                    '1hb2':['Jan 12, 2025', 'Jan 20, 2025'],
                    '1hb3': ['Jan 14, 2025', 'Jan 27, 2025']
                }
